# **IMDb Sentiment Analysis**

**This project involves building a sentiment analysis prediction model using LSTM. We used feature extraction to analyze the attributes of the IMDb reviews. This approach helps us to classify the sentiments in Positive and Negative.**

In [1]:
!pip install kaggle

In [2]:
#Importing packages

import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split

### **Dataset through Kaggle**

In [3]:
# Loading the kaggle json file
kaggle_dictionary = json.load(open("kaggle.json"))

In [4]:
kaggle_dictionary.keys()

# setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [5]:
# Downloading the dataset Zipfile from kaggle
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 66% 17.0M/25.7M [00:00<00:00, 122MB/s]
100% 25.7M/25.7M [00:00<00:00, 130MB/s]


In [6]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [7]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


### **Loading and exploring the dataset**

In [8]:
# Loading the dataset
dataset = pd.read_csv("/content/IMDB Dataset.csv")

In [9]:
dataset.shape

(50000, 2)

In [10]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
dataset.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [12]:
dataset['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [13]:
# Replacing entries of 'sentiment' column with 0 and 1
dataset.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)
dataset

<ipython-input-13-8c0a44aec6cb>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## **Data Preprocessing**

In [16]:
# split data into training data and test data
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [17]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [20]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

## **Model: Long Short Term Memory(LSTM)**

In [23]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

# compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [24]:
# Training the model
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 131s 248ms/step - accuracy: 0.7248 - loss: 0.5315 - val_accuracy: 0.7997 - val_loss: 0.4777
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 137s 246ms/step - accuracy: 0.8454 - loss: 0.3712 - val_accuracy: 0.8415 - val_loss: 0.3919
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 124s 248ms/step - accuracy: 0.8557 - loss: 0.3453 - val_accuracy: 0.8600 - val_loss: 0.3469
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 249ms/step - accuracy: 0.8791 - loss: 0.2925 - val_accuracy: 0.8438 - val_loss: 0.3805
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 124s 249ms/step - accuracy: 0.9045 - loss: 0.2424 - val_accuracy: 0.8699 - val_loss: 0.3297


### **Model Evaluation**

In [25]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 29s 90ms/step - accuracy: 0.8728 - loss: 0.3137
Test Loss: 0.3128678500652313
Test Accuracy: 0.8741000294685364


## **Building a predictive system**

In [26]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [27]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
The sentiment of the review is: positive


In [28]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
The sentiment of the review is: negative


In [29]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
The sentiment of the review is: negative
